# Assignment 9

In [1]:
from pandas import DataFrame
from DATA225utils import make_connection, dataframe_query

## The operational tables sources

In [2]:
conn_movies = make_connection(config_file = 'movie-movies.ini')
cursor_movies = conn_movies.cursor()

conn_users = make_connection(config_file = 'movie-users.ini')
cursor_users = conn_users.cursor()

conn_production_location = make_connection(config_file = 'movie-production_location.ini')
cursor_production_location = conn_production_location.cursor()

conn_date = make_connection(config_file = 'movie-date.ini')
cursor_date = conn_date.cursor()

## The dimensional model (data warehouse)

In [3]:
conn_warehouse = make_connection(config_file = 'movie-warehouse.ini')
cursor_warehouse = conn_warehouse.cursor()

conn_warehouse2 = make_connection(config_file = 'rating-warehouse.ini')
cursor_warehouse2 = conn_warehouse2.cursor()

# movie dimension

In [4]:
cursor_movies.execute('DROP TABLE IF EXISTS movie')

sql = ( """
        CREATE TABLE movie
        (
            Movie_ID INT, 
            Movie_Title VARCHAR(255), 
            Budget INT,
            RunTime INT,
            PRIMARY KEY (Movie_ID)
        )
        """
      )
cursor_movies.execute(sql)

In [5]:
sql_Movie = ( """
              INSERT INTO movie VALUES
              (862, 'Toy Story', 30000000, 81.0), 
              (8844, 'Jumanji', 65000000, 104.0), 
              (949, 'Heat', 60000000, 170.0), 
              (710, 'GoldenEye', 58000000, 130.0),
              (1408, 'Cutthroat Island', 98000000, 119.0)
              """
          )
cursor_movies.execute(sql_Movie)
conn_movies.commit()

# users dimension

In [6]:
cursor_users.execute('DROP TABLE IF EXISTS users')

sql = ( """
        CREATE TABLE users
        (
            User_ID VARCHAR(255), 
            First_Name CHAR(255),  
            Last_Name CHAR(255),      
            User_Name VARCHAR(255),
            PRIMARY KEY (User_ID)
        )
        """
      )

cursor_users.execute(sql)

In [7]:
sql_Users = ( """
              INSERT INTO users VALUES
              ('111','Joseph','Chang','joseph123'),
              ('222','Shrey','Jain','shrey01'),
              ('333','Satyaprakash','Mishra','sp17mishra'),
              ('444','Justin','Wang','justinw002'),
              ('555','Ron','Mak','ronmak145')
              """
          )
cursor_users.execute(sql_Users)
conn_users.commit()

# production_location dimension

In [8]:
cursor_production_location.execute('DROP TABLE IF EXISTS production_location')

sql = ( """
        CREATE TABLE production_location
        (
            Country_Code CHAR(255), 
            Continent CHAR(255), 
            Country CHAR(255), 
            City CHAR(255),
            PRIMARY KEY (Country_Code)
        )
        """
      )

cursor_production_location.execute(sql)

In [9]:
sql_PL = ( """
              INSERT INTO production_location VALUES
              ('USA', 'North_America', 'United_States', 'New York'),
              ('UK', 'Europe', 'Great Britain', 'London'),
              ('CHI', 'Asia', 'China', 'Beijing'),
              ('FRA', 'Europe', 'France', 'Paris')
              
              """
          )
cursor_production_location.execute(sql_PL)
conn_production_location.commit()

# date dimension

In [10]:
cursor_date.execute('DROP TABLE IF EXISTS date')

sql = ( """
        CREATE TABLE date
        (
            FullDate DATE, 
            Year INT, 
            Month INT, 
            Day INT,
            PRIMARY KEY (FullDate)
        )
        """
      )

cursor_date.execute(sql)

In [11]:
sql_date = ( """
              INSERT INTO date VALUES
              ('2001-12-06', 2001, 12, 06),
              ('1994-10-13', 1994, 10, 13),
              ('2020-11-10', 2020, 11, 10),
              ('2021-01-01', 2021, 01, 01),
              ('2000-08-26', 2000, 08, 26)
              
              """
          )
cursor_date.execute(sql_date)
conn_date.commit()

# movie_fact Table

In [12]:
cursor_warehouse.execute('DROP TABLE IF EXISTS movie_fact')

sql = ( """
        CREATE TABLE movie_fact
        (
            Movie_ID INT, 
            Country_Code CHAR(255), 
            FullDate DATE, 
            Revenue INT,
            PRIMARY KEY(Movie_ID, Country_Code, FullDate)
        )
        """
      )

cursor_warehouse.execute(sql)

In [13]:
sql_movie_fact = ( """
              INSERT INTO movie_fact VALUES
              (710, 'CHI', '2000-08-26', 4000000), 
              (862, 'USA', '1994-10-13', 20000000),
              (8844, 'FRA', '2001-12-06', 1000000000)
              
              """
          )
cursor_warehouse.execute(sql_movie_fact)
conn_warehouse.commit()

# rating_fact Table

In [14]:
cursor_warehouse2.execute('DROP TABLE IF EXISTS rating_fact')

sql = ( """
        CREATE TABLE rating_fact
        (
            User_ID INT, 
            Movie_ID INT,  
            FullDate DATE, 
            Rating FLOAT,
            PRIMARY KEY(User_ID, Movie_ID, FullDate)
        )
        """
      )

cursor_warehouse2.execute(sql)

In [15]:
sql_RF = ( """
              INSERT INTO rating_fact VALUES
              ('12', 13, '2023-11-10', 5.6), 
              ('111', 2333, '2021-01-01', 3.2), 
              ('333', 1, '2000-08-26', 9.2)
              
              """
          )
cursor_warehouse2.execute(sql_RF)
conn_warehouse2.commit()

# Queries for movies_fact

In [16]:
sql = ( """
        SELECT mm.Movie_ID, mm.Movie_Title, mm.Budget, mw.Revenue, mw.Revenue - mm.Budget AS Profit  
        FROM `movie-movies`.movie mm, `movie-warehouse`.movie_fact mw, `movie-date`.date md
        WHERE mm.Movie_ID = mw.Movie_ID
        AND   mw.FullDate = md.FullDate
        AND   md.FullDate = "1994-10-13";
        """
      )

cursor_warehouse.execute(sql)
df1 = DataFrame(cursor_warehouse.fetchall(), columns = ["Movie_ID", 'Movie_Title', 'Budget', 'Revenue', 'Profit'])
display(df1)


,Movie_ID,Movie_Title,Budget,Revenue,Profit
0,862,Toy Story,30000000,20000000,-10000000


In [17]:
# This query finds the the budget, revenue, and profit for a movie on October 13th, 1994. 
# We see that Movie ID 862 is shown.

In [18]:
sql = ( """
        SELECT mm.Movie_ID, mm.Movie_Title, mp.Country_Code, mp.City
        FROM `movie-movies`.movie mm, `movie-warehouse`.movie_fact mw, `movie-production_location`.production_location mp
        WHERE mw.Country_Code = mp.Country_Code
        AND mw.Movie_ID = mm.Movie_ID
        AND mm.Movie_Title = "Goldeneye"
        """
      )

cursor_warehouse.execute(sql)
df2 = DataFrame(cursor_warehouse.fetchall(), columns = ["Movie_ID", 'Movie_Title', 'Country', 'City'])
display(df2)


,Movie_ID,Movie_Title,Country,City
0,710,GoldenEye,CHI,Beijing


In [19]:
# This query finds the production location for the movie GoldenEye. We see it was filmed in China. 
# We also see that GoldenEye is Movie_ID 710.

# Queries for rating_fact

In [20]:
sql = ( """
        SELECT rw.User_ID, rw.Movie_ID, rw.Rating, md.FullDate
        FROM `movie-users`.users mu, `rating-warehouse`.rating_fact rw, `movie-date`.date md
        WHERE mu.User_ID = rw.User_ID
        AND   rw.FullDate = md.FullDate
        AND   rw.FullDate = "2021-01-01"
        """
      )

cursor_warehouse2.execute(sql)
df3 = DataFrame(cursor_warehouse2.fetchall(), columns = ["User_ID", 'Movie_ID', 'Rating', 'Date'])
display(df3)

,User_ID,Movie_ID,Rating,Date
0,111,2333,3.2,2021-01-01


In [21]:
# This query finds the rating given on a specific date. In this case, the date is January 1st, 2021. 
# User_ID 111 gave this rating to Movie_ID 2333.

In [22]:
sql = ( """
        SELECT rw.User_ID, rw.Movie_ID, rw.Rating
        FROM `movie-users`.users mu, `rating-warehouse`.rating_fact rw
        WHERE mu.User_ID = rw.User_ID
        AND mu.User_ID = "111"
        """
      )

cursor_warehouse2.execute(sql)
df4 = DataFrame(cursor_warehouse2.fetchall(), columns = ['User_ID', 'Movie_ID', 'Rating'])
display(df4)

,User_ID,Movie_ID,Rating
0,111,2333,3.2


In [23]:
# This query finds what are the movies rated by user 111. We see the user rated movie id 2333 as 3.2.